# Data Visualisation

In [ ]:
# import statements
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import matplotlib.pyplot as plt
import json

In [ ]:
# this line is needed for the inline display of graphs in Jupyter Notebook
%matplotlib notebook

topic = 'AssignmentTaskC'

def annotate_max(x, y, ax = None):
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = 'Max: Time={}, Value={}'.format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax, ymax+5), arrowprops=dict(facecolor='red', shrink=0.05),)
    
def annotate_min(x, y, ax = None):
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = 'Min: Time={}, Value={}'.format(xmin, ymin)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmin, ymin), xytext=(xmin, ymin+5), arrowprops=dict(facecolor='orange', shrink=0.05),)

def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=10000, # stop iteration if no message after 10 sec
                                   #auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

def init_plots():
    try:
        width = 9.5
        height = 4
        fig = plt.figure(figsize=(width,height)) # create new figure
        ax = fig.add_subplot(111) # adding the subplot axes to the given grid position
        fig.suptitle('Air Temperature Streaming') # giving figure a title
        ax.set_xlabel('Receiving Time')
        ax.set_ylabel('Air Temperature')
        ax.set_ylim(0,100) 
        ax.set_yticks([0,20,40,60,80])
        fig.show() # displaying the figure
        fig.canvas.draw() # drawing on the canvas
        return fig, ax
    except Exception as ex:
        print(str(ex))
    
def consume_messages(consumer, fig, ax):
    try:
        # container for x and y values
        x, y = [], []
        for message in consumer:
            data = message.value.decode('utf-8')

            my_data = json.loads(data)
            my_data = my_data.replace("'","\"")
            my_data = json.loads(my_data)
            receive_timestamp = dt.datetime.now().strftime("%X")
        
            sender_id = my_data['sender_id']

            if(sender_id == 'P01'):
                x.append(receive_timestamp) 
                y.append(int(my_data['air_temperature_celcius']))
                        
            # we start plotting only when we have 4 data points
            if len(y) > 4:
                ax.clear()
                ax.plot(x, y)
                ax.set_xlabel('Receiving Time')
                ax.set_ylabel('Air Temperature')
                ax.set_ylim(0,100) 
                ax.set_yticks([0,20,40,60,80])
                annotate_max(x,y)
                annotate_min(x,y)
                fig.canvas.draw()
                x.pop(0) # removing the item in the first position
                y.pop(0)
        plt.close('all')
    except Exception as ex:
        print(str(ex))
    
if __name__ == '__main__':
    
    consumer = connect_kafka_consumer()
    fig, ax = init_plots()
    consume_messages(consumer, fig, ax)
    
    

#   

# Static Data Visualisation 

##  Bar Chart

In [ ]:
import gmplot
import pymongo
import webbrowser

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["streamingDataDB"]
streams = db["streams"]

chart_data_list = []
fire_map_data_list = []

doc_id_obj_list = streams.aggregate([{"$project": {"_id":1,"length":{"$size":"$satellite_data"}}},{"$sort":{"length": -1 }},{"$limit":10}])


for doc_id_obj in doc_id_obj_list:
    doc_id = doc_id_obj["_id"]
    query_map_data = {"_id": doc_id}
    map_data = streams.find(query_map_data,{"_id":0,"create_time":1,"satellite_data":1,"relative_humidity":1,"air_temperature_celcius":1,"latitude":1,"longitude":1})
    for row in map_data:
        latitude = row["latitude"]
        longitude = row["longitude"]
        time_extracted = row["create_time"]
        surface_temperature = (row["satellite_data"][0]["surface_temperature_celcius"])
        confidence = (row["satellite_data"][0]["confidence"])
        air_temperature = row["air_temperature_celcius"]
        relative_humidity = row["relative_humidity"]
    
    chart_data_list.append((doc_id_obj["length"],time_extracted))
    fire_map_data_list.append((latitude,longitude,surface_temperature,confidence,air_temperature,relative_humidity))
    
x = []
y = []
for chart_data in chart_data_list:
    x.append(chart_data[1])
    y.append(chart_data[0])

plt.figure(figsize=(16,5))
plt.bar(x,y,align='center')
plt.xlabel('Time',fontsize = 20)
plt.ylabel('Number of Fires',fontsize = 16)      


## Fire Map

In [ ]:
gmap = gmplot.GoogleMapPlotter(-37.812015244225677, 144.951471202974, 7) # latitude and longitude of melbourne city

latitude = []
longitude = []
for fire_map_data in fire_map_data_list:
    latitude.append(float(fire_map_data[0]))
    longitude.append(float(fire_map_data[1]))
    marker_label = "surface temp: " + fire_map_data[2] + ", confidence: " + fire_map_data[3] + ", air temp:" + fire_map_data[4] + ", relative humidity: " + fire_map_data[5]
    gmap.marker(float(fire_map_data[0]),float(fire_map_data[1]),title = marker_label)
    
gmap.scatter(latitude,longitude,"#FF4500", size=2, marker=False)
gmap.draw("fire-location.html")
webbrowser.open_new("fire-location.html")